In [280]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model, model_from_json
from keras.layers import Conv2D, MaxPooling2D, InputLayer
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras.utils import to_categorical
from keras.optimizers import Adam
import cv2
import numpy as np
import tensorflow as tf
import logging
tf.get_logger().setLevel(logging.ERROR)

In [281]:
def makeImageArray(directory): 
    imgs = []
    for i in sorted(os.listdir(directory)): 
        path = os.path.join(directory, i)
        img = cv2.imread(path, cv2.COLOR_BGR2RGB)
        try: 
            img = cv2.resize(img, (128,128))
            imgs.append(img)
        except: 
            pass #jupyter hidden file throwing an error 
    return np.array(imgs)

In [282]:
def makeModelArchitecture(): 
    model = Sequential()
    model.add(Conv2D(input_shape=(128,128,3),filters=32, kernel_size=5, strides=1, padding='same',activation='relu'))
    model.add(MaxPooling2D(pool_size=5, padding='same'))

    model.add(Conv2D(filters=50, kernel_size=5, strides=1, padding='same',activation='relu'))
    model.add(MaxPooling2D(pool_size=5, padding='same'))

    model.add(Conv2D(filters=80, kernel_size=5, strides=1, padding='same',activation='relu'))
    model.add(MaxPooling2D(pool_size=5, padding='same'))

    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(512, input_shape =(None,320), activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4, input_shape =(None,512),activation='softmax'))
    optimizer = Adam(lr=1e-3)
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])
    return model

In [283]:
def convertToCategorical(y): 
    return to_categorical(y, 4)

In [284]:
choices =['front', 'inside', 'house_other', 'not_house']
def humanReadableLabel(prediction_lbl): 
    return choices[prediction_lbl]

In [285]:
def fitModel(model, train_X, train_y): 
    model.fit(x=train_X, y=train_y, epochs=50, batch_size=100)

In [286]:
def makePredictionsAndComputeAccuracy(test_X, test_y):
    predictions =[]
    results = model.predict(test_X)
    for r in results: 
        maxval = np.max(r)
        predictions.append(np.squeeze(np.where(r==maxval)[0]).item(0))
    print(len([i for i, j in zip(predictions, test_y) if i == j])/len(test_y))

In [287]:
train_dir = 'data/images/train'
train_y_sparse = [3,3,3,3,1,0,1,0,1,1,0,1,1,0,0,1,1,0,1,1,0,0,1,2,0,1,1,0,1,1,1,2,0,0,0,0,0,1,1,0,0,1,1,0,1,1,0,0,1,1,1,1,1,0,0,1,1,0,1,1,1,1,0,0,1,0,0,1,1,1,1,0,0,1,1,1,0,1,1,1,2,0,1,1,1,1,1,0,0,1,1,1,1,1,1,0,1,0,1,1,1,0,1,1]
test_dir = 'data/images/test'
test_y_sparse = [3,1,1,1,0,0,0,1,1,1,0]

train_X = makeImageArray(train_dir)
test_X = makeImageArray(test_dir)

train_y = convertToCategorical(train_y_sparse)
test_y = convertToCategorical(test_y_sparse)

model = makeModelArchitecture()
fitModel(model, train_X, train_y)

Epoch 1/50
104/104 [==============================] - 1s 12ms/step - loss: 25.6884 - accuracy: 0.3942
Epoch 2/50
104/104 [==============================] - 1s 9ms/step - loss: 16.2084 - accuracy: 0.4615
Epoch 3/50
104/104 [==============================] - 1s 9ms/step - loss: 8.2046 - accuracy: 0.5673
Epoch 4/50
104/104 [==============================] - 1s 9ms/step - loss: 5.2252 - accuracy: 0.4519
Epoch 5/50
104/104 [==============================] - 1s 9ms/step - loss: 3.7162 - accuracy: 0.5577
Epoch 6/50
104/104 [==============================] - 1s 9ms/step - loss: 3.1850 - accuracy: 0.5192
Epoch 7/50
104/104 [==============================] - 1s 9ms/step - loss: 2.4727 - accuracy: 0.5096
Epoch 8/50
104/104 [==============================] - 1s 9ms/step - loss: 2.4649 - accuracy: 0.6250
Epoch 9/50
104/104 [==============================] - 1s 9ms/step - loss: 1.6529 - accuracy: 0.6154
Epoch 10/50
104/104 [==============================] - 1s 9ms/step - loss: 1.5247 - accuracy: 0.6

In [288]:
model.summary()

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_46 (Conv2D)           (None, 128, 128, 32)      2432      
_________________________________________________________________
max_pooling2d_46 (MaxPooling (None, 26, 26, 32)        0         
_________________________________________________________________
conv2d_47 (Conv2D)           (None, 26, 26, 50)        40050     
_________________________________________________________________
max_pooling2d_47 (MaxPooling (None, 6, 6, 50)          0         
_________________________________________________________________
conv2d_48 (Conv2D)           (None, 6, 6, 80)          100080    
_________________________________________________________________
max_pooling2d_48 (MaxPooling (None, 2, 2, 80)          0         
_________________________________________________________________
dropout_31 (Dropout)         (None, 2, 2, 80)        

In [289]:
model.save('dl_keras_first_attempt.h5')

In [290]:
prev_model = load_model('dl_keras_first_attempt.h5')
prev_model.summary()

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_46 (Conv2D)           (None, 128, 128, 32)      2432      
_________________________________________________________________
max_pooling2d_46 (MaxPooling (None, 26, 26, 32)        0         
_________________________________________________________________
conv2d_47 (Conv2D)           (None, 26, 26, 50)        40050     
_________________________________________________________________
max_pooling2d_47 (MaxPooling (None, 6, 6, 50)          0         
_________________________________________________________________
conv2d_48 (Conv2D)           (None, 6, 6, 80)          100080    
_________________________________________________________________
max_pooling2d_48 (MaxPooling (None, 2, 2, 80)          0         
_________________________________________________________________
dropout_31 (Dropout)         (None, 2, 2, 80)        